In [11]:
import sys
sys.path.append("../analysis/behavior")
from fnc_analysis import *
import os

In [13]:
#load tidy data and subset dataframes for later use

# load tidy data and subset dataframes for later use
data_dir = "../data/behavior"

#load unlimited intake
df = load_data(os.path.join(data_dir,'pre_processed/unlimited_intake/total_licks_normal_intake.csv'))
dft = load_data(os.path.join(data_dir,'pre_processed/unlimited_intake/lick_times_normal_intake.csv'))

#select glucose data
#unlimited intake
df_test_g = subset_df(df, 'glucose', 'test')
dft_test_g = subset_df(dft, 'glucose', 'test')

df_train_g = subset_df(df, 'glucose', 'training')
dft_train_g = subset_df(dft, 'glucose', 'training')


#select fat data
#unlimited intake
df_test_f = subset_df(df, 'fat', 'test') 
dft_test_f = subset_df(dft, 'fat', 'test') 

df_train_f = subset_df(df, 'fat', 'training') 
dft_train_f = subset_df(dft, 'fat', 'training') 

In [16]:
# determine the amount of time post infusion before we need to take the controls bottles away
# we do this by determining a lick threshold based on the normal intake controls and figuring out the 
# average time it took the normal intake stim mice to reach that threshold

# get the average licks of the controls per session
thresh = df_train_g.loc[False,].pivot_table(index=['day','sess'], values = 'total_licks').round().total_licks
lick_times = dft_train_g.loc[True,].reset_index().set_index(['mouse','day','sess','lick_number']).time

tmp = {}
for (day,sess), th in thresh.items():
    # for each day/sess get the time post-infusion when all stim mice reached the respective threshold
    t = (lick_times.loc[:, day, sess, th-1] - lick_times.loc[:, day, sess, 19])/60 - 5
    tmp.update( {(day, sess) : t } )
    
thresh_times = pd.concat(tmp, names = ['day','sess']).unstack(['day','sess'])

# make sure we include any mice that never 
mice = lick_times.index.get_level_values('mouse').unique()
mice = mice[~mice.isin(thresh_times.index)]
empty_arr = np.ones((len(mice), thresh_times.columns.size))*np.nan
empty_df = pd.DataFrame(empty_arr, index = mice, columns = thresh_times.columns)
thresh_times = pd.concat((thresh_times, empty_df), axis =0 )

# all the nan values represent sessions where a given mouse didn't reach the threshold
# here we fill those nan values with the time from the hypothetical end of the infusion (assuming 5 min inf for limited intake)
# to the end of the 30 min sess
t_end = lick_times.groupby(['mouse','day','sess']).apply(lambda x: (1800 - x.loc[:,:,:,19].values[0])/60 - 5).unstack(['day','sess'])
thresh_times.update(t_end, filter_func = pd.isnull)
thresh_times.mean(axis=0, skipna=True)


day  sess
0    0.0     7.906119
     1.0     8.854488
1    0.0     9.791560
     1.0     6.731131
2    0.0     6.334345
     1.0     5.460476
dtype: float64

In [19]:
thresh_times.mean(axis=0, skipna=True).mean()

7.513019841269841